In [64]:
import os
class Database:
    def __init__(self, name:str="cards.db"):
        self.__name = name
        self.__connection = None
        self.__cursor = None
        self.connect()
        self.create()
        
    def get_curs(self):
        return self.__cursor
    
    def get_con(self):
        return self.__connection
    
    def get_name(self):
        return self.__name
    
    def get_name_stripped(self):
        return self.__name[:-3]
    
    def connect(self):
        self.__connection = sqlite3.connect(self.__name)
        self.__cursor = self.__connection.cursor()
        assert self.get_name() in os.listdir(), "Connection FAILED"
        
    def select(self, col:list , table:str):
        res = self.exe()(f"SELECT {','.join(col)} FROM {table}")
        return res
    
    def create(self, table="cards", cols=["front","back","current_bin","last_viewed","created"]):
        # This may cause an issue if it's the firs time on the app
        try:
            self.exe()(f"CREATE TABLE {table}({', '.join(cols)})")
            print(f"CREATED: TABLE: {table} | {cols} in {self.get_name_stripped()}")
        except:
            print(f" TABLE: {table} | {cols} in {self.get_name_stripped()}")
        
    def exe(self):
        return self.get_curs().execute
    
    def get_all_cards(self):
        return db.select(["*"],"cards").fetchall()
    
    def push_card(self, card: Card):
        self.exe()(f"""INSERT INTO {self.get_name_stripped()} VALUES
                    ('{card.front}', '{card.back}', {card.current_bin}, {card.last_viewed.timestamp()}, {str(card.created.timestamp())})""")
        self.get_con().commit()
        
    def delete_db(self):
        confirmation = input("Are you sure about this mannnn?")
        if confirmation == "y":
            os.remove(f"{self.get_name()}")
            
    def cards_from_db(self):
        out =list()
        for tup in self.get_all_cards():
            out.append(Card(tup[0],tup[1],tup[2]))
        return out


In [65]:
from datetime import datetime, timedelta
import sqlite3


class Card:
    def __init__(self,front = '', back = '', current_bin=0):
        self.front = front
        self.back  = back
        self.created = datetime.now()
        self.last_viewed = self.created   
        self.current_bin = current_bin

    def __repr__(self):
        return f"{self.front} | {self.back}"
    __str__ = __repr__

    def last_viewed(self):
        return self.last_viewed
    
    def view(self):
        self.last_viewed = datetime.now()
        inp = input(f"{self.front} | ")
        misses = 0
        miss_list = list()
        while (inp.strip().lower() != self.back) and (misses <=3):
            misses +=1 
            miss_list.append(inp.strip().lower())
            print("Try again!")
            inp = input(f"{self.front} | ")
        if misses == 4:
            print(f"The back of the card is {self.back} \nYou said {', '.join(miss_list)}")

        

        
    

In [66]:
c = Card("hello there","bonjour there")

In [67]:
def print_pile(level,amount=0):
    print(f"Level {level} " +"|"*amount)

In [68]:
db = Database()

 TABLE: cards | ['front', 'back', 'current_bin', 'last_viewed', 'created'] in cards


In [69]:
cards = [Card("good","bon"),Card("house","maison"),Card("yes","oui")]

In [73]:
db.get_all_cards()

[('Hello', 'Bonjour', 0, 1686924738.552078, 1686924738.552078),
 ('good', 'bon', 0, 1687174048.773768, 1687174048.773768),
 ('house', 'maison', 0, 1687174048.773774, 1687174048.773774),
 ('yes', 'oui', 0, 1687174048.773775, 1687174048.773775),
 ('good', 'bon', 0, 1687174293.529176, 1687174293.529176),
 ('house', 'maison', 0, 1687174293.529182, 1687174293.529182),
 ('yes', 'oui', 0, 1687174293.529184, 1687174293.529184)]